In [1]:
%matplotlib notebook

In [2]:
HIVE_SNAPSHOT = "2018-01"
END_OF_DATA = "2018-02-01 00:00:00"

In [ ]:
frame = run_hive(
"""
-- TABLE OF
with mo_edits as (
    select
        event_user_text as user,
        year(event_timestamp) as year,
        month(event_timestamp) as month,
        sum(if(wiki_db = "wikidatawiki", 0.1, 1)) as edits
    from wmf.mediawiki_history
    where
        -- REGISTERED
        event_user_is_anonymous = false and
        
        -- NON-BOT
        event_user_is_bot_by_name = false and
        not array_contains(event_user_groups, "bot") and
        
        -- NON-WMF
       event_user_text not like "%WMF%" and
        
        -- CONTENT
        page_namespace_is_content_historical = true and
        
        -- EDITS
        event_entity = "revision" and
        event_type = "create" and
        
        -- FROM THE LAST 3 MONTHS (92 DAYS)
        unix_timestamp(event_timestamp, "yyyy-MM-dd HH:mm:ss.0") 
            > (unix_timestamp("{end_of_data}") - (60 * 60 * 24 * 92)) and

        -- FROM THE LATEST SNAPSHOT
        snapshot = "{hive_snapshot}"    
    -- PER USER, PER MONTH
    group by event_user_text, year(event_timestamp), month(event_timestamp)
), 

-- TABLE OF
yr_wiki_edits as (
    select
        event_user_text as user,
        wiki_db as wiki,
        sum(if(wiki_db = "wikidatawiki", 0.1, 1)) as edits,
        max(event_timestamp) as latest_edit
    from wmf.mediawiki_history
    where
        -- REGISTERED
        event_user_is_anonymous = false and
        
        -- NON-BOT
        event_user_is_bot_by_name = false and
        not array_contains(event_user_groups, "bot") and
        
        -- NON-WMF
        event_user_text not like "%WMF%" and
        
        -- EDITS
        event_entity = "revision" and
        event_type = "create" and
        
        -- FROM THE LAST YEAR (365 DAYS)
        unix_timestamp(event_timestamp, "yyyy-MM-dd HH:mm:ss.0") 
            > (unix_timestamp("{end_of_data}") - (60 * 60 * 24 * 365)) and
        
        -- FROM THE LATEST SNAPSHOT
        snapshot = "{hive_snapshot}"
    
    -- PER USER, PER WIKI
    group by event_user_text, wiki_db
)

-- FINAL SELECT OF
select 
    recent_active.user as user,
    active_months,
    global_edits,
    home_wiki.wiki as home_wiki,
    enwiki.edits as enwiki_edits

-- USERS ACTIVE IN 2 OF 3 MONTHS
from (
    select
        user,
        sum(if(edits >= 5, 1, 0)) as active_months
    from mo_edits
    group by user
    having active_months >= 2
) recent_active

-- JOINED TO THEIR HOME WIKI
left join (
    select
    user,
    wiki,
    sum(edits) over (partition by user) as global_edits,
    row_number() over (partition by user order by edits desc, latest_edit desc) as rank
    from yr_wiki_edits
) home_wiki
on 
    recent_active.user = home_wiki.user and
    rank = 1

-- JOINED TO THEIR ENWIKI EDIT COUNT
left join
yr_wiki_edits enwiki
on 
    recent_active.user = enwiki.user and 
    enwiki.wiki = "enwiki"
""".format(
        hive_snapshot = HIVE_SNAPSHOT,
        end_of_data = END_OF_DATA
    )
)

In [18]:
frame

,user,active_months,year_edits,home_wiki,enwiki_edits
0,123Apropos,2,25.0,itwiki,2.0
1,123Apropos,2,25.0,hrwiki,2.0
2,123Apropos,2,25.0,nlwiki,2.0
3,123Apropos,2,25.0,enwiki,2.0
4,123Apropos,2,25.0,plwiki,2.0
5,123Apropos,2,25.0,eswiki,2.0
6,123Apropos,2,25.0,nowiki,2.0
7,123Apropos,2,25.0,commonswiki,2.0
8,123Apropos,2,25.0,hewiki,2.0
9,123Apropos,2,25.0,enwiki,2.0


In [ ]:
len(kowiki_edits[kowiki_edits["last_year_edits"] >= 30])

In [ ]:
plt.pyplot.hist(
    np.log10(kowiki_edits["last_year_edits"]),
    bins = (
        0, np.log10(5), np.log10(10), np.log10(25),
        np.log10(100), np.log10(500), np.log10(1000), 
        np.log10(10000), np.log10(70000))
)
plt.pyplot.title("Yearly edit distribution on the Korean Wikipedia")
plt.pyplot.xlabel("edits per year")
plt.pyplot.ylabel("number of users")
plt.pyplot.show()

In [ ]:
global_edits = run_hive(
"""
select
    event_user_text as user,
    count(*) as last_year_edits
from wmf.mediawiki_history
where 
    unix_timestamp(event_timestamp, "yyyy-MM-dd HH:mm:ss.0") 
        > unix_timestamp(current_timestamp) - (60 * 60 * 24 * 365) and
    event_user_is_bot_by_name = false and
    -- not array_contains(event_user_groups, "bot") and
    user_is_anonymous = false and
    wiki_db = "{wiki}" and
    event_entity = "revision" and
    event_type = "create" and
    snapshot = "{hive_snapshot}" 
group by event_user_text
""".format(
        wiki = WIKI,
        hive_snapshot = HIVE_SNAPSHOT
    )
)

In [ ]:
run_hive(
"""
select
    event_timestamp,
    year(event_timestamp) as year,
    month(event_timestamp) as month
from wmf.mediawiki_history
where snapshot = "2017-12"
limit 10
""")